### LCEL로 RAG 구축하기

In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_qdrant import qdrant
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

#### PyPDF로 문서에서 텍스트 추출

In [2]:
loader = PyPDFLoader(
    file_path="data/arxiv_paper.pdf",
)
docs = loader.load()
len(docs)

18

In [3]:
for doc in docs:
    print(doc.page_content[:500])
    print(doc.metadata)
    print("-"*100)

Uncertainty in Action: Confidence Elicitation in Embodied Agents
Tianjiao Yu, Vedant Shah, Muntasir Wahed, Kiet A. Nguyen, Adheesh Juvekar
Tal August, Ismini Lourentzou
University of Illinois Urbana-Champaign
{ty41,vrshah4,mwahed2,kietan2,adheesh2,taugust,lourent2}@illinois.edu
https://plan-lab.github.io/ece
Abstract
Expressing confidence is challenging for embod-
ied agents navigating dynamic multimodal en-
vironments, where uncertainty arises from both
perception and decision-making processes.
{'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-03-14T01:17:30+00:00', 'author': 'Tianjiao Yu,, Vedant Shah,, Muntasir Wahed,, Kiet A. Nguyen,, Adheesh Juvekar, Tal August,, Ismini Lourentzou', 'keywords': 'Embodied Agents, Uncertainty in Embodied AI, Verbalized Uncertainty', 'moddate': '2025-03-14T01:17:30+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': 'Proceedings of the Int

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

print(splits[0].page_content)

Uncertainty in Action: Confidence Elicitation in Embodied Agents
Tianjiao Yu, Vedant Shah, Muntasir Wahed, Kiet A. Nguyen, Adheesh Juvekar
Tal August, Ismini Lourentzou
University of Illinois Urbana-Champaign
{ty41,vrshah4,mwahed2,kietan2,adheesh2,taugust,lourent2}@illinois.edu
https://plan-lab.github.io/ece
Abstract
Expressing confidence is challenging for embod-
ied agents navigating dynamic multimodal en-
vironments, where uncertainty arises from both
perception and decision-making processes. We
present the first work investigating embodied con-
fidence elicitation in open-ended multimodal en-
vironments. We introduce Elicitation Policies,
which structure confidence assessment across
inductive, deductive, and abductive reasoning,
along with Execution Policies, which enhance
confidence calibration through scenario reinter-
pretation, action sampling, and hypothetical rea-
soning. Evaluating agents in calibration and fail-
ure prediction tasks within the Minecraft envi-


In [5]:
len([len(chunk.page_content) for chunk in splits])

100

#### Ollama Embedding으로 bge-m3 기반 텍스트 임베딩

In [6]:
embeddings = OllamaEmbeddings(model="bge-m3")

#### Qdrant 벡터DB(인메모리)에 임베딩 저장

In [7]:
from langchain_qdrant import QdrantVectorStore, RetrievalMode
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from uuid import uuid4


# 로컬 저장소를 위한 Qdrant 클라이언트 생성
client = QdrantClient(path="/tmp/arxiv_paper")

# # 밀집 벡터로 컬렉션 생성
# client.create_collection(
#     collection_name="embodied_agent_0413",
#     vectors_config=VectorParams(size=1024, distance=Distance.COSINE),
# )

qdrant = QdrantVectorStore(
    client=client,
    collection_name="embodied_agent_0413",
    embedding=embeddings,
    retrieval_mode=RetrievalMode.DENSE,
)

# uuids = [str(uuid4()) for _ in range(len(splits))]

# qdrant.add_documents(documents=splits, ids=uuids)

# 벡터 저장소를 검색기로 설정


#### 벡터DB 기반 Retriever 구현

In [8]:
retriever = qdrant.as_retriever()

search_result=retriever.invoke("Embodied Agent가 뭐야?", k=5)

for doc in search_result:
    print(doc.page_content[:500])
    print(doc.metadata)
    print("-"*100)

Uncertainty in Action: Confidence Elicitation in Embodied Agents
Tianjiao Yu, Vedant Shah, Muntasir Wahed, Kiet A. Nguyen, Adheesh Juvekar
Tal August, Ismini Lourentzou
University of Illinois Urbana-Champaign
{ty41,vrshah4,mwahed2,kietan2,adheesh2,taugust,lourent2 }@illinois.edu
https://plan-lab.github.io/ece
Abstract
Expressing confidence is challenging for embod-
ied agents navigating dynamic multimodal en-
vironments, where uncertainty arises from both
perception and decision-making processes
{'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-03-14T01:17:30+00:00', 'author': 'Tianjiao Yu,, Vedant Shah,, Muntasir Wahed,, Kiet A. Nguyen,, Adheesh Juvekar, Tal August,, Ismini Lourentzou', 'keywords': 'Embodied Agents, Uncertainty in Embodied AI, Verbalized Uncertainty', 'moddate': '2025-03-14T01:17:30+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': 'Proceedings of the Int

#### RAG 프롬프트 템플릿 설정

In [9]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
당신은 Q&A 전문 AI 어시스턴트입니다. 주어진 컨텍스트를 사용하여 질문에 답변해주세요.

컨텍스트:
{context}

질문:
{question}

답변:
""")

#### 단순 답변 Streaming하기

In [10]:
from IPython.display import Markdown
from langchain_core.runnables import RunnableParallel

llm = ChatOllama(model="gemma3:4b", temperature=0)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

for chunk in rag_chain.stream("Embodied Agent를 설명해주세요"):
    print(chunk, end='', flush=True)

제공된 컨텍스트에 따르면, Embodied Agent는 다음과 같이 설명할 수 있습니다.

*   **불확실성 획득을 위한 정책 생성:** 불확실성을 정량화하고 다양한 결과에 대비하기 위해 여러 관찰을 생성하고 다양한 행동 경로를 생성하는 정책을 생성합니다.
*   **행동 샘플링:** 현재 상태와 작업 목표에 따라 학습된 정책 분포에서 여러 가능한 행동을 생성합니다. 이를 통해 다양한 결과를 평가하고 가장 성공할 가능성이 높은 행동을 평가합니다.
*   **시나리오 재해석:** 동일한 시나리오를 다른 관점에서 재해석하여 다른 행동 경로를 제안하고 관련 환경 정보를 수집합니다.
*   **실행 정책:** 시나리오 재해석, 행동 샘플링, 가설적 추론을 통해 신뢰도 캘리브레이션을 향상시키는 정책입니다.
*   **복잡한 환경에서의 신뢰 표현:** 불확실성 평가, 추론, 환경 상호 작용을 통합하여 복잡한 환경에서 신뢰를 표현하는 데 중점을 둡니다.

요약하자면, Embodied Agent는 불확실성을 효과적으로 관리하고 표현하여 복잡한 환경에서 신뢰를 획득하고 캘리브레이션하는 데 초점을 맞춘 AI 에이전트입니다.

#### 참고한 소스를 함께 스트리밍하기

In [11]:
# 문서 컨텍스트를 기반으로 답변을 생성하는 체인 정의
rag_chain_from_docs = (
    # 입력 데이터에서 context 필드를 추출하여 format_docs 함수로 처리
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    # 포맷된 컨텍스트와 질문을 프롬프트에 전달
    | prompt
    # 프롬프트를 LLM에 전달하여 응답 생성
    | llm
    # LLM의 응답을 문자열로 변환
    | StrOutputParser()
)

# 검색 결과와 질문을 병렬로 처리하고 답변을 생성하는 최종 체인
rag_chain_with_source = RunnableParallel(
    # 검색기(retriever)로 문서를 가져오고, 질문을 그대로 전달
    {"context": retriever, "question": RunnablePassthrough()}
# 위에서 생성된 context와 question을 rag_chain_from_docs에 전달하여 answer 필드 추가
).assign(answer=rag_chain_from_docs)

for chunk in rag_chain_with_source.stream("Embodied Agent를 설명해주세요"):
    if "context" in chunk:
        print("📚 Retrieved Documents:")
        for i, doc in enumerate(chunk["context"]):
            print(f"소스 [{i+1}]: <{doc.metadata['title']}>")
            print(f"참고 내용: {doc.page_content[:100]}...\n")
        print("-" * 80)
    elif "answer" in chunk:
        print(chunk["answer"], end="", flush=True)

📚 Retrieved Documents:
소스 [1]: <Uncertainty in Action: Confidence Elicitation in Embodied Agents>
참고 내용: Confidence Elicitation in Embodied Agents
ter quantify uncertainty and anticipate divergent outcomes...

소스 [2]: <Uncertainty in Action: Confidence Elicitation in Embodied Agents>
참고 내용: Uncertainty in Action: Confidence Elicitation in Embodied Agents
Tianjiao Yu, Vedant Shah, Muntasir ...

소스 [3]: <Uncertainty in Action: Confidence Elicitation in Embodied Agents>
참고 내용: tend to yield improved confidence calibration. For instance,
MineLLM’s ECE achieves 0.32 and 0.30 pa...

소스 [4]: <Uncertainty in Action: Confidence Elicitation in Embodied Agents>
참고 내용: Confidence Elicitation in Embodied Agents
8. Impact Statement
This work advances Embodied AI by intr...

--------------------------------------------------------------------------------
제공된 컨텍스트에 따르면, Embodied Agent는 다음과 같이 설명할 수 있습니다.

*   **불확실성 획득을 위한 정책 생성:** 이 에이전트는 현재 상태와 작업 목표를 기반으로 다양한 행동 경로를 생성하여 신뢰도 평가를 촉진하는 정책 세트를 생성합니다.
